In [1]:
import itertools
import time
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

%matplotlib inline
plt.style.use('ggplot')

In [2]:
dataset = pd.read_csv("C:/Users/pc/Desktop/Mini 2/Linear Modelling/fulltrain.csv")

In [3]:
def press_statistic(y_true, y_pred, xs):
    res = y_pred - y_true
    hat = xs.dot(np.linalg.pinv(xs))
    den = (1 - np.diagonal(hat))
    sqr = np.square(res/den)
    return sqr.sum()

In [4]:
def fit_linear_reg(X,Y):
    #Fit linear regression model and return RSS and R squared values
    model_k = linear_model.LinearRegression(fit_intercept = True)
    model_k.fit(X,Y)
    RSS = mean_squared_error(Y,model_k.predict(X)) * len(Y)
    R_squared = model_k.score(X,Y)
    PRESS = press_statistic(Y,model_k.predict(X),X)
    return RSS, R_squared, PRESS

In [5]:
# # to calculate MSE for whole variables
# Y_whole = dataset["life_expectancy"]
# X_whole = dataset.drop(columns = "life_expectancy", axis = 1)
# tmp_result_whole = fit_linear_reg(X_whole,Y_whole)
# MSE = tmp_result_whole[0]/len(Y_whole)

In [6]:
#Importing tqdm for the progress bar
from tqdm import trange, tqdm_notebook

#Initialization variables
Y = dataset["life_expectancy"]
X = dataset.drop(columns = "life_expectancy", axis = 1)
k = 19 #number of total original margin
RSS_list, R_squared_list, PRESS_list, feature_list = [],[],[],[]
numb_features = []

#Looping over k = 12 to k = 15 features in X
for k in trange(10,13, desc = 'Loop...'):
    #Looping over all possible combinations: from 19 choose k
    for combo in itertools.combinations(X.columns,k):
        tmp_result = fit_linear_reg(X[list(combo)],Y)   #Store temp result 
        RSS_list.append(tmp_result[0])                  #Append lists
        R_squared_list.append(tmp_result[1])
        PRESS_list.append(tmp_result[2])
        feature_list.append(combo)
        numb_features.append(len(combo))   

#Store in DataFrame
df = pd.DataFrame({'numb_features': numb_features,'RSS': RSS_list, 'R_squared':R_squared_list,'features':feature_list,'PRESS':PRESS_list})
df.head(3)

Loop...: 100%|██████████| 3/3 [46:11<00:00, 968.28s/it] 


,numb_features,RSS,R_squared,features,PRESS
0,10,27892.048475,0.682734,"(status, adult_mortality, infant_deaths, alcoh...",28413.043728
1,10,28270.256327,0.678432,"(status, adult_mortality, infant_deaths, alcoh...",28810.969741
2,10,27588.738814,0.686184,"(status, adult_mortality, infant_deaths, alcoh...",28114.820497


In [72]:
# to calculate MSE for whole variables
Y_whole = dataset["life_expectancy"]
X_whole = dataset.drop(columns = "life_expectancy", axis = 1)
tmp_result_whole = fit_linear_reg(X_whole,Y_whole)
MSE = tmp_result_whole[0]/len(Y_whole)

In [73]:
#Initializing useful variables
m = len(Y)
#p = 19
#hat_sigma_squared = (1/(m - p -1)) * min(df['RSS'])

#Computing
df['C_p'] = df['RSS']/MSE - m+2*(df['numb_features']) # formular from Wiki
df['AIC'] = m*np.log(df['RSS']/m)+2*df['numb_features']
df['R_squared_adj'] = 1 - ( (1 - df['R_squared'])*(m-1)/(m-df['numb_features'] -1))
df

,numb_features,RSS,R_squared,features,PRESS,C_p,AIC,R_squared_adj
0,10,27892.048475,0.682734,"(status, adult_mortality, infant_deaths, alcoh...",28413.043728,1056.281650,3695.614031,0.679958
1,10,28270.256327,0.678432,"(status, adult_mortality, infant_deaths, alcoh...",28810.969741,1085.981215,3711.156786,0.675618
2,10,27588.738814,0.686184,"(status, adult_mortality, infant_deaths, alcoh...",28114.820497,1032.463622,3682.996228,0.683438
3,10,21970.801532,0.750087,"(status, adult_mortality, infant_deaths, alcoh...",22469.608982,591.303305,3420.238152,0.747900
4,10,28164.706694,0.679632,"(status, adult_mortality, infant_deaths, alcoh...",28655.602122,1077.692709,3706.840156,0.676830
5,10,28269.530269,0.678440,"(status, adult_mortality, infant_deaths, alcoh...",28798.125614,1085.924200,3711.127147,0.675627
6,10,27647.301627,0.685518,"(status, adult_mortality, infant_deaths, alcoh...",28168.951384,1037.062390,3685.443235,0.682766
7,10,27775.175817,0.684063,"(status, adult_mortality, infant_deaths, alcoh...",28299.300462,1047.103979,3690.768406,0.681299
8,10,23056.707605,0.737735,"(status, adult_mortality, infant_deaths, alcoh...",23472.650795,676.576359,3475.909836,0.735440
9,10,22396.915594,0.745240,"(status, adult_mortality, infant_deaths, alcoh...",22772.110232,624.764806,3442.405204,0.743011


In [74]:
df.to_csv("results.cvs")

In [75]:
df.sort_values(by="PRESS").head(3)

,numb_features,RSS,R_squared,features,PRESS,C_p,AIC,R_squared_adj
170709,12,14806.325615,0.831581,"(status, adult_mortality, infant_deaths, alcoh...",15200.825245,32.697796,2968.800658,0.829810
170710,12,14806.064061,0.831584,"(status, adult_mortality, infant_deaths, alcoh...",15200.926468,32.677257,2968.780273,0.829813
170815,12,14812.595592,0.831510,"(status, adult_mortality, infant_deaths, alcoh...",15206.618220,33.190159,2969.289235,0.829738


In [76]:
df.sort_values(by="C_p").head(3)

,numb_features,RSS,R_squared,features,PRESS,C_p,AIC,R_squared_adj
170710,12,14806.064061,0.831584,"(status, adult_mortality, infant_deaths, alcoh...",15200.926468,32.677257,2968.780273,0.829813
170709,12,14806.325615,0.831581,"(status, adult_mortality, infant_deaths, alcoh...",15200.825245,32.697796,2968.800658,0.829810
170815,12,14812.595592,0.831510,"(status, adult_mortality, infant_deaths, alcoh...",15206.618220,33.190159,2969.289235,0.829738


In [77]:
df.sort_values(by="AIC").head(3)

,numb_features,RSS,R_squared,features,PRESS,C_p,AIC,R_squared_adj
170710,12,14806.064061,0.831584,"(status, adult_mortality, infant_deaths, alcoh...",15200.926468,32.677257,2968.780273,0.829813
170709,12,14806.325615,0.831581,"(status, adult_mortality, infant_deaths, alcoh...",15200.825245,32.697796,2968.800658,0.829810
170815,12,14812.595592,0.831510,"(status, adult_mortality, infant_deaths, alcoh...",15206.618220,33.190159,2969.289235,0.829738


In [78]:
df.sort_values(by="R_squared_adj",ascending=False).head(3)

,numb_features,RSS,R_squared,features,PRESS,C_p,AIC,R_squared_adj
170710,12,14806.064061,0.831584,"(status, adult_mortality, infant_deaths, alcoh...",15200.926468,32.677257,2968.780273,0.829813
170709,12,14806.325615,0.831581,"(status, adult_mortality, infant_deaths, alcoh...",15200.825245,32.697796,2968.800658,0.829810
170815,12,14812.595592,0.831510,"(status, adult_mortality, infant_deaths, alcoh...",15206.618220,33.190159,2969.289235,0.829738


In [79]:
df.iloc[170710]["features"]

('status',
 'adult_mortality',
 'infant_deaths',
 'alcohol',
 'percentage_expenditure',
 'bmi',
 'under-five_deaths',
 'polio',
 'hiv/aids',
 'thinness_5-9_years',
 'income_composition_of_resources',
 'schooling')